In [2]:
import os
import pandas as pd 
import json

print(os.getcwd())

df_raw = pd.read_csv(os.path.join(os.getcwd(),'Raw Data/ROBOKOP Drug-Gene-Target Triples Virtual Screening Set.csv'))


/Users/eding/PycharmProjects/U24-ROBOKOP-Project-8-21-24/Step 1 Data Processing/ROBOKOP Virtual Screening Set


In [2]:
#####import hgnc complete set to change gene names that can be changed to protein names

json_file = os.path.join(os.path.dirname(os.getcwd()), 'ROBOKOP+DrugmechDB/hgnc_complete_set.json')
with open(json_file, 'r') as file:
    data = json.load(file)

data_len = len(data['response']['docs'])

gene_prot_list = []

for i in range(data_len):
    try:
        gene_symbol = data['response']['docs'][i]['symbol']
        protein_name = data['response']['docs'][i]['name']
        protein_id = data['response']['docs'][i]['uniprot_ids'][0]
        row = [gene_symbol,protein_name,protein_id]
        gene_prot_list.append(row)
    except KeyError:
        gene_symbol = data['response']['docs'][i]['symbol']
        protein_name = data['response']['docs'][i]['name']
        protein_id = 'null'
        row = [gene_symbol,protein_name,protein_id]
        gene_prot_list.append(row)

columns = ['symbol','name','uniprot id']

hgnc_df = pd.DataFrame(gene_prot_list, columns=columns)


In [3]:
print(hgnc_df)

         symbol                                               name uniprot id
0          A1BG                             alpha-1-B glycoprotein     P04217
1      A1BG-AS1                               A1BG antisense RNA 1       null
2          A1CF                     APOBEC1 complementation factor     Q9NQ94
3           A2M                              alpha-2-macroglobulin     P01023
4       A2M-AS1                                A2M antisense RNA 1       null
...         ...                                                ...        ...
43846    ZYG11B       zyg-11 family member B, cell cycle regulator     Q9C0D3
43847       ZYX                                              zyxin     Q15942
43848     ZYXP1                                 zyxin pseudogene 1       null
43849     ZZEF1  zinc finger ZZ-type and EF-hand domain contain...     O43149
43850      ZZZ3                   zinc finger ZZ-type containing 3     Q8IYH5

[43851 rows x 3 columns]


In [4]:
print(df_raw)

                                                  c.name          c.id  \
0                                             vandetanib   CHEBI:49960   
1                                              ponatinib   CHEBI:78543   
2                                              sorafenib   CHEBI:50924   
3                                            regorafenib   CHEBI:68647   
4                                            ruxolitinib   CHEBI:66919   
...                                                  ...           ...   
99995                                          LY3009120  CHEBI:167662   
99996                                          CH5126766   CHEBI:78825   
99997                                           GDC-0879   CHEBI:83405   
99998  N-(5-{[(5-tert-butyl-1,3-oxazol-2-yl)methyl]su...   CHEBI:91399   
99999                                           PLX-4720   CHEBI:90295   

                                                      r0 g.name  \
0      {"agent_type":not_provided,"knowledge

In [5]:
columns = ['c.name','c.id','d.name','d.id', 'g.name','g.id']
df = df_raw[columns]
df.columns = ['drug_name', 'drug.id','disease_name', 'disease.id','gene_name','gene.id']

In [6]:
print(df)

                                               drug_name       drug.id  \
0                                             vandetanib   CHEBI:49960   
1                                              ponatinib   CHEBI:78543   
2                                              sorafenib   CHEBI:50924   
3                                            regorafenib   CHEBI:68647   
4                                            ruxolitinib   CHEBI:66919   
...                                                  ...           ...   
99995                                          LY3009120  CHEBI:167662   
99996                                          CH5126766   CHEBI:78825   
99997                                           GDC-0879   CHEBI:83405   
99998  N-(5-{[(5-tert-butyl-1,3-oxazol-2-yl)methyl]su...   CHEBI:91399   
99999                                           PLX-4720   CHEBI:90295   

                               disease_name     disease.id gene_name  \
0      multiple endocrine neoplasia typ

In [7]:
for i in range(len(df)):
    df.iloc[i,0] = df.iloc[i,0].lower()
    df.iloc[i,2] = df.iloc[i,2].lower()
    df.iloc[i,4] = df.iloc[i,4].lower()

df['gene_name'] = df['gene_name'].replace('pgk 1.00', 'pgk1').replace('npr 2.00', 'npr2').replace('top 1.00', 'top1').replace('cyp 20.00', 'cyp20')

/var/folders/91/rbzkmg7n5g7f992d83ryjnz40000gn/T/ipykernel_79056/3305338610.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.iloc[i,0] = df.iloc[i,0].lower()
/var/folders/91/rbzkmg7n5g7f992d83ryjnz40000gn/T/ipykernel_79056/3305338610.py:3: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df.iloc[i,2] = df.iloc[i,2].lower()
/var/folders/91/rbzkmg7n5g7f992d83ryjnz40000gn/T/ipykernel_79056/3305338610.py:4: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view

In [35]:
#####TO OBTAIN DATASET WITH ONLY PROTEIN DESCRIPTORS (IF YOU ONLY WANT GENE NAMES SKIP THIS STEP)######

def GetProteinName(gene_name):
    try:
        i = hgnc_df[hgnc_df['symbol']==gene_name.upper()].index.values
        #print(str(i) + " is the index")
        index = int(i[0])
        protein = hgnc_df.at[index, 'name']
        protein_id = hgnc_df.at[index, 'uniprot id']
        #print(gene + " maps to " + protein)
    except:
        print(f"Could not map gene symbol:{gene_name}")
        protein = 'null'
        protein_id = 'null'
    return protein, protein_id

for i in range(len(df)):
    protein_descriptor, protein_id = GetProteinName(df.iloc[i,4])
    df.iloc[i,4] = protein_descriptor.lower()
    df.iloc[i,5] = protein_id

filtered_df = df[(df['gene_name'] != "null") & (df['gene.id'] != "null")]  ###remove all rows where gene couldnt be converted to protein name
filtered_df.rename(columns ={'gene_name':'protein_name', 'gene.id':'protein_id'},inplace = True)
filtered_df.to_csv(os.path.join(os.getcwd(), 'Processed Data/ROBOKOP Virtual Screening Set Triples.csv')
)

Could not map gene symbol:nd1
Could not map gene symbol:nd1
Could not map gene symbol:nd1
Could not map gene symbol:nd1
Could not map gene symbol:nd1
Could not map gene symbol:nd1
Could not map gene symbol:nd1
Could not map gene symbol:nd1
Could not map gene symbol:nd1
Could not map gene symbol:nd1
Could not map gene symbol:cyp20
Could not map gene symbol:cyp20
Could not map gene symbol:cyp20
Could not map gene symbol:cyp20
Could not map gene symbol:cyp20
Could not map gene symbol:cyp20
Could not map gene symbol:cyp20
Could not map gene symbol:cyp20
Could not map gene symbol:cyp20
Could not map gene symbol:cyp20
Could not map gene symbol:cyp20
Could not map gene symbol:cyp20
Could not map gene symbol:cyp20
Could not map gene symbol:cyp20
Could not map gene symbol:cyp20
Could not map gene symbol:cyp20
Could not map gene symbol:cyp20
Could not map gene symbol:cyp20
Could not map gene symbol:cyp20
Could not map gene symbol:cyp20
Could not map gene symbol:cyp20
Could not map gene symbol:cy

/var/folders/91/rbzkmg7n5g7f992d83ryjnz40000gn/T/ipykernel_90116/1242341842.py:23: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  filtered_df.rename(columns ={'gene_name':'protein_name', 'gene.id':'protein_id'},inplace = True)
